In [5]:
from datetime import datetime, date, time, timedelta
from itertools import islice
from geopy.distance import great_circle
from operator import itemgetter
import matplotlib.pylab as plt
from pandas import DataFrame
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import numpy
import json
import math


DELTA_GNSS = 0.05 #equivalent to about 5-6 km
DELTA_GNSS_INIT = 1 #equivalent to about 5-6 km
DELTA_POS = 0.0001 #equivalent to about 100 m

#six decimal are enough for a decimeter precision
#0.1 = 11.1 km
#0.01 = 1.11 km
#...
#0.000001 = 0.111 m
uav_latitude = 45.0059289
uav_longitude = 10.461025
uav_coo = (uav_latitude, uav_longitude)

#support variables
bts_latitude = 0
bts_longitude = 0

#Import UAV Log network Ping data from JSON file
data = []
with open('LORAWAN_TTGO.json') as f:
    for line in f:
        try:
            data.append(json.loads(line))
        except:
            print('Skip line')

#Convert JSON data from Python dict to DF
UAV_LOG = pd.json_normalize(data)

#Drop unusfull data
#UAV_LOG = UAV_LOG.drop(columns=['uav_param.status','uav_param.mode','interface','gps_param.groundspeed','gps_param.velocity.vx','uav_param.attitude.pitch','uav_param.attitude.yaw','uav_param.attitude.roll','gps_param.velocity.vy','gps_param.velocity.vz','gps_param.hdop','gps_param.vdop','gps_param.fix_type'])

#Remove duplicate entries
UAV_LOG = UAV_LOG.drop_duplicates('timestamp')
UAV_LOG = UAV_LOG.apply(pd.to_numeric, errors='ignore')

#Remove false flights
#UAV_bench = UAV_LOG['uav_param.battery']>=10.0
#UAV_LOG = UAV_LOG[UAV_bench]
UAV_LOG
#Remove no GPS fix data
#GPS_Null = UAV_LOG['gps_param.satellites']>=6
#UAV_LOG = UAV_LOG[GPS_Null]

#Sort by time for time clustering
#UAV_LOG = UAV_LOG.sort_values(by='timestamp')

,timestamp,gateways,rssi,snr,latitude,longitude,altitude,hdop,satellites
0,2022-07-19T07:03:47.036096458Z,"[{'gateway_ids': {'gateway_id': 'uniprgw', 'eu...",-31,13.0,45.004825,10.467922,16,0.9,8
1,2022-07-19T07:04:07.118812212Z,"[{'gateway_ids': {'gateway_id': 'uniprgw', 'eu...",-32,13.8,45.004825,10.467922,16,0.9,8
2,2022-07-19T07:04:27.211992887Z,"[{'gateway_ids': {'gateway_id': 'uniprgw', 'eu...",-31,11.0,45.004825,10.467922,16,0.9,8
3,2022-07-19T07:04:47.318032429Z,"[{'gateway_ids': {'gateway_id': 'uniprgw', 'eu...",-32,13.2,45.004825,10.467922,15,0.9,8
4,2022-07-19T07:05:07.434762820Z,"[{'gateway_ids': {'gateway_id': 'uniprgw', 'eu...",-37,13.5,45.004836,10.467836,14,0.9,8
...,...,...,...,...,...,...,...,...,...
93,2022-07-19T07:43:39.366689121Z,"[{'gateway_ids': {'gateway_id': 'uniprgw', 'eu...",-31,9.5,45.004879,10.467922,13,0.8,9
94,2022-07-19T07:43:59.900003612Z,"[{'gateway_ids': {'gateway_id': 'uniprgw', 'eu...",-42,12.5,45.004879,10.467922,12,0.8,9
95,2022-07-19T07:44:20.494950217Z,"[{'gateway_ids': {'gateway_id': 'uniprgw', 'eu...",-29,12.8,45.004847,10.467901,12,0.8,9
96,2022-07-19T07:44:41.059664030Z,"[{'gateway_ids': {'gateway_id': 'uniprgw', 'eu...",-28,10.2,45.004847,10.467922,13,0.8,9
